In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/"
RES_DIR = ROOT_DIR + "resources/"
MOD_DIR = ROOT_DIR + "ml_models/"
BATCH_SIZE = 32

X = pd.read_csv(RES_DIR + "X_NEG2.csv")["Sysnet"]
y = pd.read_csv(RES_DIR + "y_NEG2.csv")["NEG"]


In [3]:
BUFFER_SIZE = 1000
BATCH_SIZE = 128


In [4]:
X_pom, X_val, y_pom, y_val = train_test_split(X, y, test_size=.2, stratify=y, )
X_train, X_test, y_train, y_test = train_test_split(X_pom, y_pom, test_size=.1, stratify=y_pom, )

X_val = tf.convert_to_tensor(X_val, name ="Definicija")
y_val = tf.convert_to_tensor(y_val, name ="Sentiment")


X_train = tf.convert_to_tensor(X_train, name ="Definicija")
y_train = tf.convert_to_tensor(y_train, name ="Sentiment")

X_test = tf.convert_to_tensor(X_test, name ="Definicija")
y_test = tf.convert_to_tensor(y_test, name ="Sentiment")

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))


In [5]:
tf.math.confusion_matrix(labels=y_train, predictions=y_train).numpy()


array([[14214,     0],
       [    0,   222]], dtype=int32)

In [6]:
train_dataset =train_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
validation_dataset= validation_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [8]:
tf.shape(X_train)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([14436], dtype=int32)>

In [9]:
import tensorflow_datasets as tfds
import numpy as np


In [10]:
VOCAB_SIZE = 25000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode="int" )
encoder.adapt(train_dataset.map(lambda text, label: text))



In [11]:
vocab = np.array(encoder.get_vocabulary())
vocab[:30]


array(['', '[UNK]', 'koji', 'se', 'i', 'u', 'ili', 'na', 'jesam', 'od',
       'za', 'sa', 'iz', 'da', 'neki', 'imati', 'koristiti', 'biljka',
       'nešto', 'kao', 'rod', 'jezik', 'velik', 'biti', 'card',
       'porodica', 'jedan', 'mali', 'deo', 'drugi'], dtype='<U21')

In [12]:
len(encoder.get_vocabulary())

17003

In [13]:
model_NEG = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [14]:
model_NEG.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])

In [15]:
history = model_NEG.fit(train_dataset, epochs=10,
                    validation_data=validation_dataset,
                    validation_steps=30)

Epoch 1/10
113/113 [==============================] - 161s 1s/step - loss: 0.4383 - binary_accuracy: 0.9799 - val_loss: 0.1528 - val_binary_accuracy: 0.9841
Epoch 2/10
113/113 [==============================] - 111s 989ms/step - loss: 0.1459 - binary_accuracy: 0.9845 - val_loss: 0.1234 - val_binary_accuracy: 0.9841
Epoch 3/10
113/113 [==============================] - 108s 947ms/step - loss: 0.1218 - binary_accuracy: 0.9846 - val_loss: 0.1090 - val_binary_accuracy: 0.9841
Epoch 4/10
113/113 [==============================] - 109s 958ms/step - loss: 0.1126 - binary_accuracy: 0.9845 - val_loss: 0.1006 - val_binary_accuracy: 0.9841
Epoch 5/10
113/113 [==============================] - 108s 956ms/step - loss: 0.1035 - binary_accuracy: 0.9846 - val_loss: 0.0905 - val_binary_accuracy: 0.9841
Epoch 6/10
113/113 [==============================] - 109s 964ms/step - loss: 0.0816 - binary_accuracy: 0.9863 - val_loss: 0.0821 - val_binary_accuracy: 0.9859
Epoch 7/10
113/113 [=======================

In [16]:
model_NEG.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 128)         2176384   
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        263168    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        164352    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 64)         41216     
 nal)                                                   

In [17]:
y_pred = model_NEG.predict(X_test)

51/51 [==============================] - 16s 69ms/step


In [18]:
# model.save(MOD_DIR + "nntest3")

In [19]:
y_pred[:10]

array([[2.8917917e-05],
       [1.4252464e-05],
       [1.1392097e-02],
       [1.4572190e-05],
       [1.4626955e-04],
       [1.6755757e-05],
       [1.8017046e-05],
       [3.6474841e-04],
       [3.0139781e-04],
       [1.8910570e-04]], dtype=float32)

In [20]:
y_pred = tf.round((y_pred))

In [21]:
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()


In [22]:
con_mat

array([[1557,   22],
       [  12,   13]], dtype=int32)

In [23]:
y_pred = model_NEG.predict(X_val)

126/126 [==============================] - 12s 95ms/step


In [24]:
con_mat = tf.math.confusion_matrix(labels=y_val, predictions=y_pred).numpy()


In [25]:
con_mat

array([[3949,    0],
       [  62,    0]], dtype=int32)

In [26]:
X = pd.read_csv(RES_DIR + "X_POS2.csv")["Sysnet"]
y = pd.read_csv(RES_DIR + "y_POS2.csv")["POS"]

In [27]:
X_pom, X_val, y_pom, y_val = train_test_split(X, y, test_size=.1, stratify=y, )
X_train, X_test, y_train, y_test = train_test_split(X_pom, y_pom, test_size=.2, stratify=y_pom, )

X_val = tf.convert_to_tensor(X_val, name ="Definicija")
y_val = tf.convert_to_tensor(y_val, name ="Sentiment")


X_train = tf.convert_to_tensor(X_train, name ="Definicija")
y_train = tf.convert_to_tensor(y_train, name ="Sentiment")

X_test = tf.convert_to_tensor(X_test, name ="Definicija")
y_test = tf.convert_to_tensor(y_test, name ="Sentiment")

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))


In [28]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [29]:
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode="int" )
encoder.adapt(train_dataset.map(lambda text, label: text))



In [30]:
model_POS = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [31]:
model_POS.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])

In [32]:
history = model_POS.fit(train_dataset, epochs=10,
                    validation_data=validation_dataset,
                    validation_steps=30)

Epoch 1/10
113/113 [==============================] - ETA: 0s - loss: 0.4323 - binary_accuracy: 0.9830

113/113 [==============================] - 158s 1s/step - loss: 0.4323 - binary_accuracy: 0.9830 - val_loss: 0.1447 - val_binary_accuracy: 0.9865
Epoch 2/10
113/113 [==============================] - 98s 868ms/step - loss: 0.1361 - binary_accuracy: 0.9864
Epoch 3/10
113/113 [==============================] - 98s 867ms/step - loss: 0.1161 - binary_accuracy: 0.9866
Epoch 4/10
113/113 [==============================] - 102s 907ms/step - loss: 0.1106 - binary_accuracy: 0.9866
Epoch 5/10
113/113 [==============================] - 100s 888ms/step - loss: 0.0946 - binary_accuracy: 0.9867
Epoch 6/10
113/113 [==============================] - 103s 913ms/step - loss: 0.0803 - binary_accuracy: 0.9870
Epoch 7/10
113/113 [==============================] - 101s 894ms/step - loss: 0.0639 - binary_accuracy: 0.9883
Epoch 8/10
113/113 [==============================] - 100s 883ms/step - loss: 0.0507 - binary_accuracy: 0.9904
Epoch 9/10
113/113 [==============================] - 102s 906ms/step - loss: 0

In [33]:
model_POS.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         2154112   
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 256)        263168    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, None, 128)        164352    
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 64)         41216     
 nal)                                                 

In [34]:
y_pred = model_POS.predict(X_test)

113/113 [==============================] - 21s 78ms/step


In [35]:
# model.save(MOD_DIR + "nntest3")

In [36]:
y_pred[:10]

array([[2.76629347e-04],
       [3.69345695e-02],
       [6.37024204e-05],
       [6.50108705e-05],
       [1.09947316e-04],
       [6.25846744e-01],
       [3.07548908e-04],
       [6.33910531e-05],
       [1.78280316e-04],
       [1.20008124e-04]], dtype=float32)

In [37]:
y_pred = tf.round((y_pred))

In [38]:
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()


In [39]:
con_mat

array([[3536,   25],
       [  31,   17]], dtype=int32)

In [40]:
y_pred = model_POS.predict(X_val)

63/63 [==============================] - 7s 107ms/step


In [41]:
con_mat = tf.math.confusion_matrix(labels=y_val, predictions=y_pred).numpy()


In [42]:
con_mat

array([[1979,    0],
       [  27,    0]], dtype=int32)

In [43]:
def polarity_correction(pos, neg):
  one = tf.convert_to_tensor(1.0)
  ret_pos = pos*(one - neg)
  ret_neg =neg * (one -pos)
  return ret_pos, ret_neg
  

In [44]:
sword = pd.read_csv(RES_DIR +"definicije_lematizone.csv", index_col=0)

In [45]:
definicije = sword["Definicija"]

In [46]:
tensor_def = tf.convert_to_tensor(definicije.to_numpy(dtype='str'))

In [47]:
tn_POS = model_POS.predict(tensor_def)
tn_NEG = model_NEG.predict(tensor_def)


792/792 [==============================] - 63s 79ms/step


In [48]:
tn_POSc, tn_NEGc = polarity_correction(tn_POS, tn_NEG)

In [49]:
sword["POS"] = tn_POSc.numpy()

In [50]:
sword["NEG"] = tn_NEGc.numpy()

In [51]:
sword

,ID,Definicija,POS,NEG
0,ENG30-03574555-n,zgrada u koji se nalaziti organizacioni jedini...,0.000078,0.000019
1,ENG30-07810907-n,pripremljen dodatak jela za poboljšanje ukus,0.000169,0.000081
2,ENG30-00721431-n,"u nečiji prilika , mogućnost",0.000267,0.000498
3,ENG30-00473799-v,ostati jesam još samo da se doterati neki fine...,0.000092,0.000874
4,ENG30-00903385-v,zapad on jesam oprostiti što se nekada računat...,0.000102,0.001918
...,...,...,...,...
25315,ENG30-15266265-n,mandat predsednik,0.033278,0.086363
25316,ENG30-15266685-n,vreme između početak i kraj vremenski perioda,0.000124,0.000037
25317,ENG30-15266911-n,tačka u vreme kada se nešto završavati,0.000118,0.000143
25318,ENG30-15276642-n,( računarstvo ) brzina prenos podatak ( npr . ...,0.000106,0.000043


In [52]:
sword.to_csv(RES_DIR + "sentiment_RNN2.csv")